# Data Augmentation

**About the data:** <br>
The dataset contains 2 folders: yes and no which contains 253 Brain MRI Images. The folder yes contains 155 Brain MRI Images that are tumorous and the folder no contains 98 Brain MRI Images that are non-tumorous. You can find [here](https://www.kaggle.com/navoneel/brain-mri-images-for-brain-tumor-detection).

Since this is a small dataset, I used data augmentation in order to create more images.

Also, we could solve the data imbalance issue (since 61% of the data belongs to the tumorous class) using data augmentation.

## Import Necessary Modules

In [3]:
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import cv2
import imutils
import matplotlib.pyplot as plt
from os import listdir
import time    


%matplotlib inline

In [4]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m}:{round(s,1)}"

In [5]:
def augment_data(file_dir, n_generated_samples, save_to_dir):
    """
    Arguments:
        file_dir: A string representing the directory where images that we want to augment are found.
        n_generated_samples: A string representing the number of generated samples using the given image.
        save_to_dir: A string representing the directory in which the generated images will be saved.
    """
    
    #from keras.preprocessing.image import ImageDataGenerator
    #from os import listdir
    
    data_gen = ImageDataGenerator(rotation_range=10, 
                                  width_shift_range=0.1, 
                                  height_shift_range=0.1, 
                                  shear_range=0.1, 
                                  brightness_range=(0.3, 1.0),
                                  horizontal_flip=True, 
                                  vertical_flip=True, 
                                  fill_mode='nearest'
                                 )

    
    for filename in listdir(file_dir):
        # load the image
        image = cv2.imread(file_dir + '\\' + filename)
        # reshape the image
        image = image.reshape((1,)+image.shape)
        # prefix of the names for the generated sampels.
        save_prefix = 'aug_' + filename[:-4]
        # generate 'n_generated_samples' sample images
        i=0
        for batch in data_gen.flow(x=image, batch_size=1, save_to_dir=save_to_dir, 
                                           save_prefix=save_prefix, save_format='jpg'):
            i += 1
            if i > n_generated_samples:
                break

Remember that 61% of the data (155 images) are tumorous. And, 39% of the data (98 images) are non-tumorous.<br>
So, in order to balance the data we can generate 9 new images for every image that belongs to 'no' class and 6 images for every image that belongs the 'yes' class.<br>

In [12]:
import os
from keras_preprocessing.image import ImageDataGenerator
import cv2

# Define your base paths
original_data_path = 'path/to/original/data/'  # Replace with your actual path
augmented_data_path = '/home/ghost/Desktop/Brain-Tumor-Detection-master/augmented data'  # Changed space to underscore for better practice

# Define the specific paths
yes_path = os.path.join(original_data_path, '/home/ghost/Desktop/Brain-Tumor-Detection-master/augmented data/yes')  # Path to tumor images
no_path = os.path.join(original_data_path, '/home/ghost/Desktop/Brain-Tumor-Detection-master/augmented data/no')    # Path to non-tumor images

# Create the augmented directories if they don't exist
os.makedirs(os.path.join(augmented_data_path, 'yes'), exist_ok=True)
os.makedirs(os.path.join(augmented_data_path, 'no'), exist_ok=True)

# Define your augmentation function
def augment_data(file_dir, n_generated_samples, save_to_dir):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )
    
    for filename in os.listdir(file_dir):
        image = cv2.imread(os.path.join(file_dir, filename))
        if image is None:
            continue
            
        image = image.reshape((1,) + image.shape)
        save_prefix = 'aug_' + os.path.splitext(filename)[0]
        
        i = 0
        for batch in datagen.flow(image, batch_size=1, save_to_dir=save_to_dir,
                                 save_prefix=save_prefix, save_format='jpg'):
            i += 1
            if i > n_generated_samples:
                break

# Now call the function with the defined paths
augment_data(file_dir=yes_path, n_generated_samples=6, save_to_dir=os.path.join(augmented_data_path, 'yes'))
augment_data(file_dir=no_path, n_generated_samples=9, save_to_dir=os.path.join(augmented_data_path, 'no'))

Let's see how many tumorous and non-tumorous examples after performing data augmentation:

In [13]:
def data_summary(main_path):
    
    yes_path = main_path+'yes'
    no_path = main_path+'no'
        
    # number of files (images) that are in the the folder named 'yes' that represent tumorous (positive) examples
    m_pos = len(listdir(yes_path))
    # number of files (images) that are in the the folder named 'no' that represent non-tumorous (negative) examples
    m_neg = len(listdir(no_path))
    # number of all examples
    m = (m_pos+m_neg)
    
    pos_prec = (m_pos* 100.0)/ m
    neg_prec = (m_neg* 100.0)/ m
    
    print(f"Number of examples: {m}")
    print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {m_pos}") 
    print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {m_neg}") 

In [16]:
import os
from os import listdir

# Define your base paths correctly
main_path = '/home/ghost/Desktop/Brain-Tumor-Detection-master/augmented data/'  # Note the trailing slash
yes_path = os.path.join(main_path, 'yes')  # Proper path joining
no_path = os.path.join(main_path, 'no')    # Proper path joining

def data_summary(main_path):
    # Verify paths exist first
    if not os.path.exists(yes_path):
        raise FileNotFoundError(f"Directory not found: {yes_path}")
    if not os.path.exists(no_path):
        raise FileNotFoundError(f"Directory not found: {no_path}")
    
    # Count files
    m_pos = len([f for f in os.listdir(yes_path) if os.path.isfile(os.path.join(yes_path, f))])
    m_neg = len([f for f in os.listdir(no_path) if os.path.isfile(os.path.join(no_path, f))])
    
    # Number of all examples
    m = (m_pos + m_neg)
    
    # Porportion
    pos_prec = (m_pos*100.0)/m
    neg_prec = (m_neg*100.0)/m
    
    print(f"Number of examples: {m}")
    print(f"Percentage of positive examples: {pos_prec:.2f}%")
    print(f"Percentage of negative examples: {neg_prec:.2f}%")

# Call the function
data_summary(main_path)

Number of examples: 19455
Percentage of positive examples: 44.61%
Percentage of negative examples: 55.39%


That's it for this notebook. Now, we can use the augmented data to train our convolutional neural network.